In [66]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas import Series

#importing the dataset using pandas
df = pd.read_csv("./911.csv")

#sample of original dataset
df.head(10)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1
5,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,2015-12-10 15:39:04,LANSDALE,CANNON AVE & W 9TH ST,1
6,40.182111,-75.127795,LAUREL AVE & OAKDALE AVE; HORSHAM; Station 35...,19044.0,EMS: NAUSEA/VOMITING,2015-12-10 16:46:48,HORSHAM,LAUREL AVE & OAKDALE AVE,1
7,40.217286,-75.405182,COLLEGEVILLE RD & LYWISKI RD; SKIPPACK; Stati...,19426.0,EMS: RESPIRATORY EMERGENCY,2015-12-10 16:17:05,SKIPPACK,COLLEGEVILLE RD & LYWISKI RD,1
8,40.289027,-75.399590,MAIN ST & OLD SUMNEYTOWN PIKE; LOWER SALFORD;...,19438.0,EMS: SYNCOPAL EPISODE,2015-12-10 16:51:42,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,1
9,40.102398,-75.291458,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD; PLYM...,19462.0,Traffic: VEHICLE ACCIDENT -,2015-12-10 17:35:41,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,1


In [67]:
#separate timeStamp data into 2 new columns
df['date'] = df.timeStamp.str[0:11]
df['time'] = df.timeStamp.str[-8:]

#Get rid of dummy 'e' column and 'timeStamp' column
del df['e']
del df['timeStamp']
#If time at end, then try to extract station number and impute
del df['desc']

#sample of dataset
#df.head(10)

In [68]:
#count number of values in each column
df.count()

lat      177755
lng      177755
zip      155957
title    177755
twp      177694
addr     177755
date     177755
time     177755
dtype: int64

In [69]:
#Finds total number of rows with missing values 
#Rows with more than one missing value only counted once
df.isnull().any(axis=1).sum()

21822

In [70]:
#Drop rows with missing values
df = df.dropna(subset=['zip', 'twp'])
df.count()

lat      155933
lng      155933
zip      155933
title    155933
twp      155933
addr     155933
date     155933
time     155933
dtype: int64

In [71]:
#df.head(10)

In [72]:
hour = df.time.str[0:2]
hour2 = pd.to_numeric(hour)

#If time of call is between 6PM and 6AM then it is classified as 'night', otherwise it is classified as 'day'

for i, row in df.iterrows():
    if(hour2.loc[i] >= 18 or hour2.loc[i] < 6):
        hour.at[i] = 'night'
    else:
        hour.at[i] = 'day'


In [73]:
#Replace military time with either 'night' or 'day'
del df['time']

df['time_of_day'] = hour

#df.head(10)   

In [74]:
#Change date format to weekdays format
df['dates'] = pd.to_datetime(df['date'])
df['weekday'] = df['dates'].dt.weekday_name

del df['date']
del df['dates']

In [75]:
#Separate first part of 911 call classification from rest of title
df['class'], df['title2'] = df['title'].str.split(':', 1).str
del df['title']
del df['title2']


In [76]:
df.head(10)

,lat,lng,zip,twp,addr,time_of_day,weekday,class
0,40.297876,-75.581294,19525.0,NEW HANOVER,REINDEER CT & DEAD END,day,Thursday,EMS
1,40.258061,-75.264680,19446.0,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,day,Thursday,EMS
2,40.121182,-75.351975,19401.0,NORRISTOWN,HAWS AVE,day,Thursday,Fire
3,40.116153,-75.343513,19401.0,NORRISTOWN,AIRY ST & SWEDE ST,day,Thursday,EMS
5,40.253473,-75.283245,19446.0,LANSDALE,CANNON AVE & W 9TH ST,day,Thursday,EMS
6,40.182111,-75.127795,19044.0,HORSHAM,LAUREL AVE & OAKDALE AVE,day,Thursday,EMS
7,40.217286,-75.405182,19426.0,SKIPPACK,COLLEGEVILLE RD & LYWISKI RD,day,Thursday,EMS
8,40.289027,-75.399590,19438.0,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,day,Thursday,EMS
9,40.102398,-75.291458,19462.0,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,day,Thursday,Traffic
11,40.084161,-75.308386,19428.0,PLYMOUTH,BROOK RD & COLWELL LN,day,Thursday,Traffic


In [77]:
#unique zipcodes in the dataset
df.zip.unique()

array([ 19525.,  19446.,  19401.,  19044.,  19426.,  19438.,  19462.,
        19428.,  19040.,  19027.,  18936.,  18974.,  19031.,  19403.,
        19422.,  19085.,  18964.,  19038.,  19406.,  19468.,  19010.,
        19095.,  19464.,  19444.,  19041.,  19440.,  19405.,  19002.,
        19096.,  19454.,  19465.,  19004.,  19066.,  19072.,  18041.,
        19046.,  19090.,  19012.,  19025.,  19473.,  18073.,  18969.,
        18074.,  19460.,  19001.,  18054.,  19009.,  19006.,  19035.,
        19150.,  19075.,  19034.,  19151.,  19453.,  19003.,  18914.,
        19512.,  18976.,  19120.,  18915.,  18076.,  19477.,  19087.,
        18966.,  19131.,  19128.,  19083.,  19053.,  19475.,  18960.,
        19504.,  18070.,  19492.,  18932.,  19118.,  18092.,  19490.,
        19518.,  18056.,  19119.,  19107.,  19111.,  18927.,  19435.,
        18951.,  19472.,  19503.,  19126.,  19505.,  19423.,  19138.,
        36107.,  18036.,  19116.,  19139.,  19129.,  19115.,  19355.,
        77316.,  194

In [78]:
#find the number of unique zipcodes
s = Series(df.zip)
zip_unique = s.unique().size
print(zip_unique)

108


In [79]:
#unique townships in the dataset
df.twp.unique()

array(['NEW HANOVER', 'HATFIELD TOWNSHIP', 'NORRISTOWN', 'LANSDALE',
       'HORSHAM', 'SKIPPACK', 'LOWER SALFORD', 'PLYMOUTH',
       'UPPER MORELAND', 'CHELTENHAM', 'MONTGOMERY', 'WHITEMARSH',
       'UPPER GWYNEDD', 'LOWER PROVIDENCE', 'WHITPAIN', 'DELAWARE COUNTY',
       'FRANCONIA', 'WEST CONSHOHOCKEN', 'UPPER MERION', 'LIMERICK',
       'DOUGLASS', 'LOWER MERION', 'POTTSTOWN', 'BRIDGEPORT', 'TOWAMENCIN',
       'AMBLER', 'LOWER POTTSGROVE', 'CHESTER COUNTY', 'UPPER HANOVER',
       'SPRINGFIELD', 'ROCKLEDGE', 'ABINGTON', 'WEST NORRITON',
       'ROYERSFORD', 'UPPER DUBLIN', 'UPPER SALFORD', 'CONSHOHOCKEN',
       'PENNSBURG', 'TELFORD', 'EAST NORRITON', 'UPPER FREDERICK',
       'UPPER PROVIDENCE', 'SALFORD', 'LEHIGH COUNTY', 'MARLBOROUGH',
       'BRYN ATHYN', 'LOWER MORELAND', 'HATBORO', 'LOWER GWYNEDD',
       'WORCESTER', 'COLLEGEVILLE', 'SCHWENKSVILLE', 'SOUDERTON',
       'PERKIOMEN', 'LOWER FREDERICK', 'BUCKS COUNTY', 'RED HILL',
       'WEST POTTSGROVE', 'UPPER POTTSGROV

In [80]:
#find the number of unique townships
s2 = Series(df.twp)
twp_unique = s2.unique().size
print(twp_unique)

68


In [81]:
#find the number of unique address locations
s4 = Series(df.addr)
s4.unique().size

25155